In [8]:
import pandas as pd
import numpy as np

In [10]:
# Dataset original
df = pd.read_csv('data\esets_lung_exprs_genes.txt',sep="\t",header=None,index_col=0)

In [4]:
# Código de Jorge
df2=df.transpose()
df2.columns=df2.columns.fillna('sample_name')
df2['sample_name']=df2['sample_name'].str[:9]
df3=pd.read_csv('data\E-MTAB-6699.sdrf.txt', sep="\t",header=0,index_col=False)
df4=df3.drop(['Extract Name','Factor Value[disease]'],axis=1)
df5=df3.drop(df4,axis=1)
df5['Tumor']=np.where(df5['Factor Value[disease]']=='tumor',1 ,0)
df6=df5.drop(['Factor Value[disease]'],axis=1)
null_columns4=df2.columns[df2.isna().any()]
df2[null_columns4].isna().sum()
df7=df2.merge(df6,how='inner',left_on='sample_name',right_on='Extract Name',left_index=True)
df7.set_index(df7['sample_name'],inplace=True)
df7=df7.drop(['sample_name','Extract Name'],axis=1)
df_caract=df7.drop('Tumor',axis=1)
df_result=df7.drop(df_caract,axis=1)

In [11]:
# Separación de data de entrenamiento 
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df_caract,df_result,test_size=0.1,random_state=42)

In [12]:
# Revisar el tema de los indices, se dropean para probar.
X_train = X_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
X_train.sample(3)

,RFC2,HSPA6,PAX8,GUCA1A,THRA,PTPN21,CCL5,CYP2E1,EPHB3,ESRRA,...,NAF1,CCDC63,C16orf52,LOC101929475,LOC389641,LOC400499,GALR3,NUS1P3,TMEM231,LOC100505915
805,7.32119867922262,9.04142449450112,8.0502678582012,5.81615126587793,7.04843607859051,7.37808216393539,10.3137321618555,5.55375953095846,7.46279468168173,7.80610494967705,...,6.56912,4.32497,3.48958,4.67824,4.55405,6.5036,5.87382,3.32329,6.45261,5.48111
157,8.06790975589815,4.61664685608931,7.6394967849001,5.77903938860889,7.39958792579677,6.84233218027575,7.21295935046614,7.27190417394847,7.22840658257563,8.37243121155485,...,6.29661,5.10372,3.94315,4.4387,5.60586,6.49402,6.06832,3.96267,6.28983,4.85883
726,8.80391774747977,7.31530352613562,7.63819454592978,5.79097468553756,6.67729587969628,11.2240264562287,7.84980856387603,5.14699739031755,7.28518241521597,7.19611420308093,...,7.07207,4.82039,4.52811,4.57713,4.48568,6.07608,6.38921,4.81714,5.20644,5.74792


In [ ]:
# Balanceamos la data antes de que entre a nuestra red neuronal para que los pesos no aumenten
from sklearn.preprocessing import StandardScaler  

scaler = StandardScaler()  
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test) 

In [ ]:
# Ejecutamos una red de multiclases y hallamos el score
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,activation=relu, hidden_layer_sizes=(2, 2500), random_state=1)
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

In [ ]:
from sklearn.metrics import cohen_kappa_score

y_predict = clf.predict(X_train)

tup_y_predict = list(y_predict)
tup_y_train = list(np.transpose(y_train['Tumor']))

tup_y_predict = [int(i) for i in tup_y_predict]

cohen_kappa_score(tup_y_train,tup_y_predict)